# Deep-Learning with Keras on Amazon SageMaker with ANN from the Superdatascience course

See: https://www.superdatascience.com/courses/deep-learning-az

In [35]:
!pip install pydot

You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Step 1: Download the data set 
The data sets are needed to train our Neural Network. We use the Churn_Modelling data set from the Superdatascience courses

In [2]:
!wget https://s3-ap-southeast-1.amazonaws.com/sgavinio-machine-learning-datasets/Churn_Modelling.csv

--2019-03-13 12:11:08--  https://s3-ap-southeast-1.amazonaws.com/sgavinio-machine-learning-datasets/Churn_Modelling.csv
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.32.105
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.32.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/csv]
Saving to: ‘Churn_Modelling.csv.1’

Churn_Modelling.csv 100%[===================>] 668.81K  --.-KB/s    in 0.01s   

2019-03-13 12:11:08 (60.5 MB/s) - ‘Churn_Modelling.csv.1’ saved [684858/684858]



## Step 2.1: Pre-process the data

Read the data set and encode categorical features.

In [36]:
# Start pre-processing template
import numpy as np
import pandas as pd

dataset = pd.read_csv("Churn_Modelling.csv")
features = dataset[[
        "CreditScore", 
        "Geography", 
        "Gender", 
        "Age", 
        "Tenure", 
        "Balance", 
        "NumOfProducts", 
        "HasCrCard", 
        "IsActiveMember", 
        "EstimatedSalary"
]]
output = dataset[["Exited"]]

# Convert dataframes to Arrays
X = features.values
y = output.values

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Encode the categorical features: Gender and Geography
labelencoder_geography = LabelEncoder()
labelencoder_gender = LabelEncoder()
X[:, 1] = labelencoder_geography.fit_transform(X[:, 1])
X[:, 2] = labelencoder_gender.fit_transform(X[:, 2])
# We only one-hot-encode the geography, the age only has two values and will result in a dummy variable trap
# See n - 1 dummy variable rule
onehotencoder_geography = OneHotEncoder(categorical_features = [1])
X = onehotencoder_geography.fit_transform(X).toarray()
# We eliminate the first dummy variable from the geography: n - 1 dummy variable rule
X = X[:, 1:]

## Step 2.2: Split the data set into the training and test sets

In [37]:
# Split the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Step 2.3: Feature Scale the dependent variables

In [38]:
# Feature scale
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Step 3: Push the pre-processed data set to S3

Push the the processed train and test data sets to S3 such that they can be consumed easier during the training phase.

In [6]:
# if this produces an error, don't sweat it. Just means you already have the directory.
!mkdir ./data

mkdir: cannot create directory ‘./data’: File exists


In [39]:
%%time

import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

sess = sagemaker.Session()
s3_bucket = sess.default_bucket()
s3_prefix = 'churn_modeling'

arn:aws:iam::572101368213:role/service-role/AmazonSageMaker-ExecutionRole-20181228T215310
CPU times: user 117 ms, sys: 30.2 ms, total: 147 ms
Wall time: 3.24 s


In [40]:
np.save('./data/train_X.npy', X_train)
np.save('./data/train_Y.npy', y_train)
np.save('./data/test_X.npy', X_test)
np.save('./data/test_Y.npy', y_test)

In [41]:
data_s3_prefix = '{}/data'.format(s3_prefix)

In [48]:
data_dir = sess.upload_data(path='./data/', bucket=s3_bucket, key_prefix=data_s3_prefix)

## Step 4: Build the Artificial Neural Network

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [ ]:
def keras_model_fn(hyperparameters):
    # Define the layers
    classifier = Sequential()
    # relu is rectifier activation function
    # Dense is simple fully-connected layer
    classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 11, name = "inputs"))
    # Second hidden layer, no need to define inpout_dim, this layer is automatically linked to the first layer
    classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu"))
    # Output layer
    classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))
    # Compile the ANN, configure back-propagation
    # adam is a type of stochastic gradient descent, binary_crossentropy is used for binary outputs
    classifier.compile(optimizer = hyperparameters["optimizer"], loss = "binary_crossentropy", metrics = ["accuracy"])

    return classifier

In [ ]:
classifier=keras_model_fn({"optimizer": "adam"})
SVG(model_to_dot(classifier,  show_shapes=True, show_layer_names=True, rankdir='HB').create(prog='dot', format='svg'))

## Step 5: Train on the notebook instance to verify

As a sanity check before launching a SageMaker training job, we train the model on the notebook instance to check that all is well.

In [ ]:
classifier.summary()
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, verbose = 1)

## Step 6: Train as a SageMaker training job

The TensorFlow estimator uses the `keras_ann.py` script as the entrypoint. Give special attention to the `keras_model_fn` which was re-defined within this python script.

In [57]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(
    entry_point="keras_ann.py",
    role=role,
    framework_version="1.12.0",
    train_instance_count=1, 
    train_instance_type="ml.c4.xlarge",
    training_steps=10000, 
    evaluation_steps=1000,
    hyperparameters={'optimizer': 'adam'}
)
tf_estimator.fit(data_dir)

INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-2019-03-13-23-45-59-596


2019-03-13 23:46:02 Starting - Starting the training job...
2019-03-13 23:46:03 Starting - Launching requested ML instances......
2019-03-13 23:47:08 Starting - Preparing the instances for training......
2019-03-13 23:48:25 Downloading - Downloading input data
2019-03-13 23:48:25 Training - Training image download completed. Training in progress.
2019-03-13 23:48:25,415 INFO - root - running container entrypoint
2019-03-13 23:48:25,415 INFO - root - starting train task
2019-03-13 23:48:25,436 INFO - container_support.training - Training starting
2019-03-13 23:48:28,601 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2019-03-13 23:48:28,601 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2019-03-13 23:48:28,601 INFO - tf_container - ---------------------------------------------------------
2019-03-13 23:48:28,601 INFO - tf_container - creating RunConfig:
2019-03-13 23:48:28,601

2019-03-13 23:48:41,858 INFO - tensorflow - global_step/sec: 305.187
2019-03-13 23:48:41,858 INFO - tensorflow - loss = 0.3997276, step = 901 (0.328 sec)
2019-03-13 23:48:42,199 INFO - tensorflow - global_step/sec: 292.924
2019-03-13 23:48:42,200 INFO - tensorflow - loss = 0.39758265, step = 1001 (0.341 sec)
2019-03-13 23:48:42,500 INFO - tensorflow - global_step/sec: 332.589
2019-03-13 23:48:42,500 INFO - tensorflow - loss = 0.39607325, step = 1101 (0.301 sec)
2019-03-13 23:48:42,820 INFO - tensorflow - global_step/sec: 312.226
2019-03-13 23:48:42,821 INFO - tensorflow - loss = 0.39509162, step = 1201 (0.320 sec)
2019-03-13 23:48:43,136 INFO - tensorflow - global_step/sec: 316.884
2019-03-13 23:48:43,136 INFO - tensorflow - loss = 0.39426276, step = 1301 (0.316 sec)
2019-03-13 23:48:43,537 INFO - tensorflow - global_step/sec: 249.462
2019-03-13 23:48:43,537 INFO - tensorflow - loss = 0.39343664, step = 1401 (0.401 sec)
2019-03-13 23:48:43,845 INFO - tensorflow - global_step/sec: 324.3

2019-03-13 23:49:01,854 INFO - tensorflow - global_step/sec: 308.212
2019-03-13 23:49:01,854 INFO - tensorflow - loss = 0.3269983, step = 7101 (0.324 sec)
2019-03-13 23:49:02,233 INFO - tensorflow - global_step/sec: 264.045
2019-03-13 23:49:02,233 INFO - tensorflow - loss = 0.32697815, step = 7201 (0.379 sec)
2019-03-13 23:49:02,545 INFO - tensorflow - global_step/sec: 320.419
2019-03-13 23:49:02,545 INFO - tensorflow - loss = 0.32696253, step = 7301 (0.312 sec)
2019-03-13 23:49:02,869 INFO - tensorflow - global_step/sec: 308.673
2019-03-13 23:49:02,869 INFO - tensorflow - loss = 0.3269482, step = 7401 (0.324 sec)
2019-03-13 23:49:03,181 INFO - tensorflow - global_step/sec: 319.921
2019-03-13 23:49:03,182 INFO - tensorflow - loss = 0.32693473, step = 7501 (0.313 sec)
2019-03-13 23:49:03,497 INFO - tensorflow - global_step/sec: 316.866
2019-03-13 23:49:03,497 INFO - tensorflow - loss = 0.32692268, step = 7601 (0.316 sec)
2019-03-13 23:49:03,809 INFO - tensorflow - global_step/sec: 320.0


2019-03-13 23:49:23 Uploading - Uploading generated training model
2019-03-13 23:49:23 Completed - Training job completed
Billable seconds: 71


### Optional: Train using TensorFlow ScriptMode

The above SageMaker training invocation executes the training and validation using the TensorFlow Legacy mode. To run using the newer ScriptMode, execute the following cell instead.

See: https://docs.aws.amazon.com/sagemaker/latest/dg/tf.html

In [56]:
# Based-off: https://github.com/aws-samples/amazon-sagemaker-script-mode/blob/master/keras-embeddings-script-mode/keras-embeddings.ipynb

from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(
    entry_point="keras_ann_script_mode.py",
    role=role,
    framework_version="1.12.0",
    train_instance_count=1, 
    train_instance_type="ml.c4.xlarge",
    hyperparameters={'learning_rate': 0.1, 
                     'epochs': 100, 
                     'batch_size': 10},
    script_mode=True
)

tf_estimator.fit({'train': data_dir, 'eval': data_dir})

INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-scriptmode-2019-03-13-23-39-06-606


2019-03-13 23:39:09 Starting - Starting the training job...
2019-03-13 23:39:14 Starting - Launching requested ML instances......
2019-03-13 23:40:17 Starting - Preparing the instances for training...
2019-03-13 23:41:09 Downloading - Downloading input data
2019-03-13 23:41:09 Training - Downloading the training image..
2019-03-13 23:41:16,644 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-03-13 23:41:16,651 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-03-13 23:41:17,103 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-03-13 23:41:17,119 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-03-13 23:41:17,130 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "module_dir": "s3://sagemaker-ap-southeast-1-572101368213/sagemaker-tensorflow-scriptmode-2019-03-13-23-39-06-606/source/sourcedir.tar.gz", 
    "channel_input_dirs"


2019-03-13 23:41:13 Training - Training image download completed. Training in progress.#015  10/8000 [..............................] - ETA: 2s - loss: 0.6173 - acc: 0.7000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 330/8000 [>.............................] - ETA: 1s - loss: 0.5277 - acc: 0.7818#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 640/8000 [=>............................] - ETA: 1s - loss: 0.5268 - acc: 0.7828#010#010#

#015  10/8000 [..............................] - ETA: 1s - loss: 0.5038 - acc: 0.8000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 320/8000 [>.............................] - ETA: 1s - loss: 0.4975 - acc: 0.8031#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 600/8000 [=>............................] - ETA: 1s - loss: 0.4927 - acc: 0.8067#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

#015  10/8000 [..............................] - ETA: 2s - loss: 0.3774 - acc: 0.9000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 320/8000 [>.............................] - ETA: 1s - loss: 0.4598 - acc: 0.8281#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 630/8000 [=>............................] - ETA: 1s - loss: 0.4620 - acc: 0.8270#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

...

#015  10/8000 [..............................] - ETA: 1s - loss: 0.5021 - acc: 0.8000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 320/8000 [>.............................] - ETA: 1s - loss: 0.4649 - acc: 0.8250#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 640/8000 [=>............................] - ETA: 1s - loss: 0.4778 - acc: 0.8172#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

#015  10/8000 [..............................] - ETA: 1s - loss: 0.6316 - acc: 0.7000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 320/8000 [>.............................] - ETA: 1s - loss: 0.4893 - acc: 0.8125#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 630/8000 [=>............................] - ETA: 1s - loss: 0.4909 - acc: 0.8095#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

#015  10/8000 [..............................] - ETA: 1s - loss: 0.5037 - acc: 0.8000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 320/8000 [>.............................] - ETA: 1s - loss: 0.4873 - acc: 0.8125#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 640/8000 [=>............................] - ETA: 1s - loss: 0.5168 - acc: 0.7953#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#


2019-03-13 23:43:41 Uploading - Uploading generated training model#015  10/8000 [..............................] - ETA: 1s - loss: 0.2099 - acc: 1.0000#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 320/8000 [>.............................] - ETA: 1s - loss: 0.4524 - acc: 0.8344#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 640/8000 [=>............................] - ETA: 1s - loss: 0.4857 - acc: 0.8141#010#010#010#010#010#010#010#0


2019-03-13 23:43:46 Completed - Training job completed
Billable seconds: 172
